In [2]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here


#Initial Load

austin311 = requests.get("https://data.austintexas.gov/resource/xwdj-i9he.json?$limit=50000&$where=sr_created_date>'2021-03-24'" ) 
austin311json = austin311.json()
cols = ['sr_number','sr_created_date','sr_closed_date','sr_type_desc','sr_location', 'sr_location_zip_code','sr_location_city','sr_status_desc','sr_location_lat','sr_location_long']
austin_df = pd.DataFrame(austin311json, columns=cols)
austin_df

In [3]:
#Incremental load
austin_table_id = "opendatadbt.311.austin311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(sr_created_date), max(sr_closed_date)
FROM `opendatadbt.311.austin311`
limit 10
"""

austinmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
austinmaxopendatedate = str(austinmaxopendatedataframe['f0_'][0])

austinmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
austinmaxclosedate = str(austinmaxclosedatedataframe['f1_'][0])


In [4]:

austin311 = requests.get("https://data.austintexas.gov/resource/xwdj-i9he.json?$limit=50000&$where=sr_created_date>'"+austinmaxopendatedate+"' OR sr_closed_date between '" + austinmaxclosedate+ "' AND '" + current_datetime + "'") 
austin311json = austin311.json()
cols = ['sr_number','sr_created_date','sr_closed_date','sr_type_desc','sr_location', 'sr_location_zip_code','sr_location_city','sr_status_desc','sr_location_lat','sr_location_long']
austin_df = pd.DataFrame(austin311json, columns=cols)
austin_df

,sr_number,sr_created_date,sr_closed_date,sr_type_desc,sr_location,sr_location_zip_code,sr_location_city,sr_status_desc,sr_location_lat,sr_location_long
0,21-00154240,2021-03-31T11:29:12.000,2021-04-08T09:11:51.000,Road Markings/Striping - New,"1222 BROADMOOR DR, AUSTIN, TX 78723",78723,AUSTIN,Open,30.31023606,-97.70563856
1,21-00169606,2021-04-08T08:06:35.000,2021-04-08T09:11:51.000,ARR Missed Recycling,"13406 TIERRA DR, AUSTIN, TX 78727",78727,AUSTIN,New,30.4347273,-97.70661914
2,21-00169608,2021-04-08T08:07:20.000,2021-04-08T09:11:51.000,Debris in Street,"800 ROSEDALE TER A, AUSTIN, TX 78704",78704,AUSTIN,Open,30.24142587,-97.74543059
3,21-00169610,2021-04-08T08:07:43.000,2021-04-08T08:15:55.000,Austin Code - Request Code Officer,"10207 BROWNIE DR, AUSTIN, TX",78753,AUSTIN,Closed,30.36711599,-97.68464536
4,21-00169613,2021-04-08T08:08:15.000,2021-04-08T09:11:51.000,Loose Dog,"4800 PROCK LN, AUSTIN, TX 78721",78721,AUSTIN,Open,30.26943466,-97.68959229
...,...,...,...,...,...,...,...,...,...,...
190,21-00169688,2021-04-08T08:49:22.000,2021-04-08T09:11:51.000,Traffic Signal - Maintenance,"SHOAL CREEK BLVD & W ANDERSON LN, AUSTIN, TX",78757,AUSTIN,Open,30.3610191,-97.74084629
191,21-00169678,2021-04-08T08:43:06.000,2021-04-08T08:52:09.000,Austin Code - Request Code Officer,"2001 MATTHEWS LN, AUSTIN, TX 78745",78745,AUSTIN,Closed,30.19990742,-97.80785745
192,21-00169679,2021-04-08T08:43:08.000,2021-04-08T08:52:11.000,Austin Code - Request Code Officer,"7603 GAINES MILL LN, AUSTIN, TX 78745",78745,AUSTIN,Closed,30.19817411,-97.81394115
193,21-00169672,2021-04-08T08:40:06.000,2021-04-08T09:11:51.000,Loose Dog,"2001 E 22ND ST, AUSTIN, TX 78722",78722,AUSTIN,Open,30.28276022,-97.72034407


In [51]:
austin_table_id = "opendatadbt.311.austin311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('austin311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    austin_df, table_ref, job_config=job_config
).result()  # Make an API request.


In [14]:
austin_table_id = "opendatadbt.311.austin311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
count(*), current_date(), "Austin"
FROM `opendatadbt.311.austin311`

"""

austindatacount = (
    client.query(query_string)
    .result()
    .to_dataframe()
)

austindatacount = austindatacount.rename(columns={'f0_':'count','f1_':'date','f2_':'city'})

austin_table_id = "opendatadbt.311.extractrowcount"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('extractrowcount')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client.load_table_from_dataframe(
    austindatacount, table_ref, job_config=job_config
).result()  # Make an API request.
